In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import csv
import os
import shutil
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Data selection
* batch 1- 5
* \> 270 days

In [ ]:
def check_duration(root):
  dur_dict = {}
  for filename in os.listdir(root):
    if filename[-4:] == '.csv':
      df = pd.read_csv(root + filename)
      if len(df) == 0:
        # os.remove(root + filename)
        print(filename, "is empty")
      else:
        # print(df)
        cgm_df = df[['displayTime', 'glucoseValue']]

        start = cgm_df["displayTime"][0]
        end = cgm_df["displayTime"][len(df)-1]
        start_date, _ = start.split('T')
        end_date, _ = end.split('T')

        # start and end date
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')

        # duration
        duration = (end_date - start_date).days + 1
        dur_dict.update({filename[:-4]: duration})
  return dur_dict

In [ ]:
# old files
batch1_root = './baseline_data/batch1/csv_cgm/'
batch2_root = './baseline_data/batch2/csv_cgm/'
# new files
batch3_root = './3rdbatch_new2_processed/csv_cgm/'
batch4_root = './4thbatch_processed/csv_cgm/'
batch5_root = './5thbatch_processed/csv_cgm/'

dst_dir = './csv_cgm/'
root_list = [batch1_root, batch2_root, batch3_root, batch4_root, batch5_root]

In [ ]:
for root in root_list:
  print(root)
  dur_dict = check_duration(root)
  selected = {k:v for k, v in dur_dict.items() if v > 30 * 9}
  print(len(selected.keys()))
  print(selected)
  print("================================")

  for i in selected.keys():
    shutil.copy(root + i + '.csv', dst_dir)

In [ ]:
count = 0
for filename in os.listdir(dst_dir):
  if filename[-4:] == '.csv':
    count += 1
print('total number of subjects:', count)

# CGM count by hour and by day

In [ ]:
cgm_dir = './csv_cgm/'
for filename in os.listdir(cgm_dir):
  if filename[-4:] == '.csv':
    cgm_df = pd.read_csv(cgm_dir + filename)[['displayTime', 'glucoseValue']]

    # drop values that have exactly same displayTime, (only keep the first one)
    cgm_df['displayTime'] = pd.to_datetime(cgm_df['displayTime'])
    cgm_df.drop_duplicates(subset=['displayTime'], inplace=True)
    cgm_df.dropna(subset=['displayTime'], inplace=True)

    # count by hours
    hours_df = cgm_df.groupby(pd.Grouper(key='displayTime', freq='H')).count()
    hours_df['Valid'] = [1 if i > 12 * .7 else 0 for i in hours_df.glucoseValue.values] # more than 9 records per hour
    hours_df['Hour']  = [pd.to_datetime(i).hour for i in hours_df.index.values]
    hours_df.rename(columns={hours_df.columns.values[0]: "Count"}, inplace = True)
    hours_df.to_csv('../cgm_count_by_hour/' + filename)

    # break

In [ ]:
cgm_dir = './csv_cgm/'
for filename in os.listdir(cgm_dir):
  if filename[-4:] == '.csv':
    cgm_df = pd.read_csv(cgm_dir + filename)[['displayTime', 'glucoseValue']]

    # drop values that have exactly same displayTime, (only keep the first one)
    cgm_df['displayTime'] = pd.to_datetime(cgm_df['displayTime'])
    cgm_df.drop_duplicates(subset=['displayTime'], inplace=True)
    cgm_df.dropna(subset=['displayTime'], inplace=True)

    # count by days
    days_df = cgm_df.groupby(pd.Grouper(key='displayTime', freq='D')).count()
    # valid if more than 202 records in that day
    days_df['Valid'] = [1 if i > 12 * 24 * .7 else 0 for i in days_df.glucoseValue.values]
    days_df.rename(columns={days_df.columns.values[0]: "Count"}, inplace = True)
    month = pd.DatetimeIndex(days_df.index).month
    day = pd.DatetimeIndex(days_df.index).day
    days_df.insert(0, 'Month', month)
    days_df.insert(1, 'Day', day) # obtain month and date of individuals

    days_df.to_csv('./cgm_count_by_day/' + filename)

    # break

In [ ]:
count = 0
hour_dir = './cgm_count_by_hour/'
for filename in os.listdir(hour_dir):
  if filename[-4:] == '.csv':
    count += 1
print('cgm_count_by_hour: total number of subjects:', count)

count = 0
day_dir = '../cgm_count_by_day/'
for filename in os.listdir(day_dir):
  if filename[-4:] == '.csv':
    count += 1
print('cgm_count_by_day: total number of subjects:', count)

# CGM summary

* Subject, start, end, duration, days with record, missing days

In [ ]:
cgm_count_dir = './cgm_count_by_day/'

summary_dict = {}

for filename in os.listdir(cgm_count_dir):
  if filename[-4:] == '.csv':
    count_df = pd.read_csv(cgm_count_dir + filename)

    start = count_df['displayTime'].values[0].split(' ')[0]
    end = count_df['displayTime'].values[-1].split(' ')[0]
    count_df['displayTime'] = pd.to_datetime(count_df['displayTime'])

    duration = pd.to_timedelta(count_df.displayTime.values[-1] - count_df.displayTime.values[0]).days + 1
    record = np.sum(count_df['Count'].values != 0)
    missing = np.sum(count_df['Count'].values == 0)
    valid = np.sum(count_df['Valid'].values == 1)

    summary_dict.update({filename[:-4]: [start, end, duration, record, missing, valid]})
    # break

In [ ]:
cgm_summary_df = pd.DataFrame(summary_dict.values(), columns=['Start', 'End', 'Duration', 'RecordDays', 'MissingDays', 'ValidDays'])
cgm_summary_df.insert(0, "Subject", summary_dict.keys())
sorted = cgm_summary_df.sort_values(by=['Duration'])

# key is ordered by duration
sorted.insert(1, "Key", [i for i in range(1, sorted.shape[0] + 1)])
sorted.head(3)

In [ ]:
sorted.to_csv('./CGM_summary.csv', index=None)

# Demographic Summary

In [ ]:
demo_file = './demographic.csv'
demo_df = pd.read_csv(demo_file)
demo_df.head(3)

In [ ]:
print(demo_df.columns.values)

cgm_df = pd.read_csv('./CGM_summary.csv')
valid_id = cgm_df['Subject'].values
print(len(valid_id))

In [ ]:
study_id_usr_dict = {}
usr_study_id_dict = {}

for i in valid_id:
  if i in demo_df.app_id.values:
    study_id_usr_dict.update({demo_df[demo_df.app_id == i].sweetgoals_id.values[0]: i})
    usr_study_id_dict.update({i: demo_df[demo_df.app_id == i].sweetgoals_id.values[0]})
    # break

In [ ]:
print("the number of valid users have demographic info:", len(study_id_usr_dict.values()))

In [ ]:
# record the used data for all valid users
valid_dict = {i:[] for i in valid_id}

for i in range(len(demo_df.sweetgoals_id.values)):
  if demo_df.app_id.values[i] in valid_id:
    # sweetgoals_id
    valid_dict[demo_df.app_id.values[i]].append(usr_study_id_dict[demo_df.app_id.values[i]])
    # gender
    valid_dict[demo_df.app_id.values[i]].append(demo_df.gender_redcap.values[i])
    # a1c
    if not np.isnan(demo_df.a1c_0.values[i]):
      valid_dict[demo_df.app_id.values[i]].append(demo_df.a1c_0.values[i])
    else:
      valid_dict[demo_df.app_id.values[i]].append('')
    # age
    if not np.isnan(demo_df.age_redcap.values[i]):
      valid_dict[demo_df.app_id.values[i]].append(demo_df.age_redcap.values[i])
    else:
      valid_dict[demo_df.app_id.values[i]].append('')
    # Race
    if not pd.isnull(demo_df.race.values[i]):
      valid_dict[demo_df.app_id.values[i]].append(demo_df.race.values[i])
    else:
      valid_dict[demo_df.app_id.values[i]].append('')
    # Diabetes Duration (Yrs.)
    if not np.isnan(demo_df.diagnosis_duration.values[i]):
      valid_dict[demo_df.app_id.values[i]].append(demo_df.diagnosis_duration.values[i])
    else:
      valid_dict[demo_df.app_id.values[i]].append('')
    # Insulin Delivery Method (e.g., multiple daily injection vs. pump treated)
    if not pd.isnull(demo_df.insulin_type.values[i]):
      valid_dict[demo_df.app_id.values[i]].append(demo_df.insulin_type.values[i])
    else:
      valid_dict[demo_df.app_id.values[i]].append('')

In [ ]:
location_df = pd.read_excel('./demographic_data/state.xlsx')

for i in range(len(location_df.StudyID.values)):
  # print(i)
  if location_df.StudyID.values[i] in study_id_usr_dict.keys():
    valid_dict[study_id_usr_dict[location_df.StudyID.values[i]]].append(location_df.State.values[i])

In [ ]:
miss_loc_id = [k for k,v in valid_dict.items() if len(v) < 8]


In [ ]:
for i in range(len(demo_df.sweetgoals_id.values)):
  # print(i)
  if demo_df.app_id.values[i] in miss_loc_id:
    valid_dict[demo_df.app_id.values[i]].append(demo_df.state.values[i])

In [ ]:
record_df = pd.DataFrame(valid_dict.values(), columns=['sweetgoals_id', 'gender', 'A1c', 'age', 'race', 'diagnosis_duration', 'insulin_type', 'location'])
record_df.insert(0, "app_id", valid_dict.keys())
record_df.head(3)

In [ ]:
record_df.to_csv('./demographic_data_summary_new.csv', index=None)

## location count

In [ ]:
location_df = pd.read_csv('./demographic_data_summary_final.csv')['location']
location_df.head(3)

In [ ]:
# https://medium.com/@jl_ruiz/plot-maps-from-the-us-census-bureau-using-geopandas-and-contextily-in-python-df787647ef77

!pip install geopandas

import geopandas as gpd

path = './tl_2022_us_state/tl_2022_us_state.shp'

df_map = gpd.read_file(path)
df_map = df_map.to_crs("EPSG:4326")

In [ ]:
stusps_name_dict = {}
for i in range(len(df_map['STUSPS'].values)):
  stusps_name_dict.update({df_map['STUSPS'].values[i]: df_map['NAME'].values[i]})

# print(stusps_name_dict)

In [ ]:
state_dict = {i:0 for i in df_map.NAME.values}
for i in location_df.values:
  if i in state_dict.keys():
    state_dict[i] += 1
  elif i in stusps_name_dict.keys():
    state_dict[stusps_name_dict[i]] += 1


print(state_dict)
print(np.sum(list(state_dict.values()))) 

In [ ]:
state_lst = [[k, v] for k, v in state_dict.items()]
state_df = pd.DataFrame(state_lst, columns=['State', 'Count'])
state_df.to_csv('./state_count_final.csv', index=None)

In [ ]:
location_df = pd.read_csv('./demographic_data_summary_final.csv')['location']
location_df.head(3)

# https://medium.com/@jl_ruiz/plot-maps-from-the-us-census-bureau-using-geopandas-and-contextily-in-python-df787647ef77

!pip install geopandas

import geopandas as gpd

path = './tl_2022_us_state/tl_2022_us_state.shp'

df_map = gpd.read_file(path)
df_map = df_map.to_crs("EPSG:4326")

df_map.head(3)

stusps_name_dict = {}
for i in range(len(df_map['STUSPS'].values)):
  stusps_name_dict.update({df_map['STUSPS'].values[i]: df_map['NAME'].values[i]})

print(stusps_name_dict)

state_dict = {i:0 for i in df_map.NAME.values}
for i in location_df.values:
  if i in state_dict.keys():
    state_dict[i] += 1
  elif i in stusps_name_dict.keys():
    state_dict[stusps_name_dict[i]] += 1


print(state_dict)
print(np.sum(list(state_dict.values()))) # all participants come from 49 US mainland

state_lst = [[k, v] for k, v in state_dict.items()]
state_df = pd.DataFrame(state_lst, columns=['State', 'Count'])
state_df.to_csv('./state_count_final.csv', index=None)